In [1]:
import sys
sys.path.append("../src")

import requests

from tonpy import LiteClient, Cell, get_block_info, BlockId, BlockIdExt, Address, Emulator, begin_cell
from tonpy.utils.shard_account import shard_is_ancestor, shard_child
from tonpy.autogen.block import Account, Transaction, Block, BlockInfo, BlockExtra, MessageAny, AccountState


# Ignore some logs from ton monorepo
from tonpy.libs.python_ton import globalSetVerbosity
globalSetVerbosity(0)

In [2]:
# Example hashes:
#   27 txs in 1 block: 184C0C0EF74E279E63B363FCC81168AEB4C3769E4991FAF6982A5E232EE88541
#   After merge: 7A074921055E014CE6B5E7C929B12D9789F5419E771F1BD98C379A57DA15064A
#   TickTock: F8EAC78DA7005B67B4A473CB77CF90ACD4A31B883E0E6C2806ADB4C0D2B3E616

In [3]:
tx_hash = "184C0C0EF74E279E63B363FCC81168AEB4C3769E4991FAF6982A5E232EE88541"

In [4]:
def get_tx_lt(tx_hash):
    query = '''query{transactions(hash: "%s"){lt,address,workchain,shard}}''' % tx_hash

    response = requests.post("https://dton.io/graphql", json={'query': query})
    
    data = response.json()['data']['transactions'][0]
    return int(data['lt']), int(data['workchain']), str(data['address']), str(data['shard'])

In [5]:
# You can not trust this data from dton.io, but IF it's correct it'll be prooved by `get_transactions` later
tx_lt, wc, address, shard = get_tx_lt(tx_hash)
block_lt = tx_lt - tx_lt % 1000000

tx_account = Address(f"{wc}:{address}")

In [6]:
# Get random LiteServer
lc = LiteClient.get_one()

# When you get transaction - you automatically proof validness by tx_hash
transactions = lc.get_transactions(tx_account, tx_lt, tx_hash, 16).transactions

# We need to get all TXs in this block
whitelist_txs = [transactions[0]]
get_state_from = None
cur_index = 0

if len(transactions) > 1:
    done = transactions[0].blkid != transactions[1].blkid
else:
    done = True
    
while not done:
    cur_index += 1
    if cur_index > len(transactions) - 1:
        # Get new 16txs to load
        transactions.extend(lc.get_transactions(tx_account, 
                                          whitelist_txs[-1].prev_trans_lt, 
                                          whitelist_txs[-1].prev_trans_hash, 16).transactions)
    if cur_index > len(transactions):
        break
    
    if whitelist_txs[-1].blkid == transactions[cur_index].blkid:
        whitelist_txs.append(transactions[cur_index])
    else:
        done = True
        

In [7]:
# If there >1 TX in block - we need to emulate all of them before our one
print("To emulate, before our TX: ", len(whitelist_txs) - 1)

whitelist_txs = list(reversed(whitelist_txs))

To emulate, before our TX:  26


In [8]:
# Get block with prev state
# WARNING: this block is not proofed
state_block = whitelist_txs[0].blkid
current_block = lc.get_block_header(state_block)
print("Current block: ", current_block.blk_id, "\n")

# We need to get master block, so we can download libs and config
if wc != -1:
    # tlb python autogen
    block = Block().cell_unpack(current_block.virt_blk_root)
    block_info = BlockInfo().cell_unpack(block.info, True)
    
    # get master block id from WC
    master = BlockIdExt(id_=BlockId(-1, 0x8000000000000000, block_info.master_ref.master.seq_no), 
          file_hash=int(block_info.master_ref.master.file_hash, 2), 
          root_hash=int(block_info.master_ref.master.root_hash, 2))
else:
    master = current_block.blk_id

print("Master block: ", master, "\n")


Current block:  (0,8000000000000000,41341632):00421695EEF1F99B1237382C3AF75A24A9C030D4439A7D33A1369A42A6544068:C580B44DEF34FA3CCE302FF888639CDB28C5845875DD4A28DC63E405E4DF3910 

Master block:  (-1,8000000000000000,35509189):57F17494F7E835E9CA1B8F72293BAC12F92B5B8BC9DD25AC3528C8D77EB61CC1:0DBC1334D1008A4AA3CB548C6495058A1B0274C0783F0F681E4FCBA6012ED2E0 



In [9]:
# Need to get rand_seed for block and get prev block with account state
current_full_block = lc.get_block(current_block.blk_id)

# It's stored in blockExtra
block = Block().cell_unpack(current_full_block)
block_info = BlockInfo().cell_unpack(block.info, True)
block_extra = BlockExtra().cell_unpack(block.extra, False)

rand_seed = int(block_extra.rand_seed, 2)

In [10]:
# To get prev block with account state - need to calculate correct shard (with account) if block is after merge
# If not after merge - just get prev seqno
if block_info.after_merge:    
    left = block_info.prev_ref.prev1
    left_shard = BlockIdExt(BlockId(wc, shard_child(current_block.blk_id.id.shard, True), left.seq_no),
                           root_hash=int(left.root_hash,2), file_hash=int(left.file_hash, 2))
    
    right = block_info.prev_ref.prev2
    right_shard = BlockIdExt(BlockId(wc, shard_child(current_block.blk_id.id.shard, False), right.seq_no),
                           root_hash=int(right.root_hash,2), file_hash=int(right.file_hash, 2))
    
    if shard_is_ancestor(left_shard.id.shard, tx_account.shard_prefix(60)):
        state_block = left_shard
    else:
        state_block = right_shard
else:
    current = block_info.prev_ref.prev
    state_block = BlockIdExt(BlockId(wc, current_block.blk_id.id.shard, current.seq_no),
                           root_hash=int(current.root_hash,2), file_hash=int(current.file_hash, 2))

In [11]:
print("State block: ", state_block, "\n")

State block:  (0,8000000000000000,41341631):5BC3E316CE6DD6E5139D1A63878BE42ED409C270F9DC05E19D2B23E5894B03C1:C0C9D1B7758391EB618B920B12344DFD5E2DACBFB8D4BF1EAC1F550B414963BC 



In [12]:
# Load account state from prev block
account_state = lc.get_account_state(f"{wc}:{address}", state_block)

# Convert answer to AccountShardState
account_state = begin_cell() \
    .store_ref(account_state.root) \
    .store_uint(int(account_state.last_trans_hash, 16), 256) \
    .store_uint(account_state.last_trans_lt, 64).end_cell()

In [13]:
# Load config of prev key block of MC block where TX was
# WARNING: this key block have no proof
key_block, config = lc.get_config_all(master, from_not_trusted_keyblock=True)

In [14]:
print("Latest key block: ", key_block)

Latest key block:  (-1,8000000000000000,35494831):91B837E3B52FF9B3F92290584605D525633280CAD34150298BAC1CE91950E07E:84C5CC5B5B9A7007E1BFEA78F3AA82A82B19681E61E274DA2211727C5BDF9207


In [15]:
# for emulation you need to get 16 latest mc blocks + 1 key block
old_mc_blocks = []
cur_seqno = master.id.seqno

for i in range(16):
    cur_seqno -= 1
    old_mc_blocks.append(lc.lookup_block(BlockId(-1, 0x8000000000000000, cur_seqno)).blk_id)

In [16]:
prev_block_data = [
    [i.to_data() for i in old_mc_blocks],
    key_block.to_data()
]

In [17]:
# Here we need to get all public libraries (todo: rewrite on new LS API if it'll be published)
# Alternative: deep_library_search 
def get_public_libs(block_hash):
    query = '''query{blocks(root_hash: "%s"){libs_hash}}''' % block_hash

    response = requests.post("https://dton.io/graphql", json={'query': query})
    
    data = response.json()['data']['blocks'][0]
    return data['libs_hash']

In [18]:
public_libs = get_public_libs(master.root_hash)

print("Found: ", len(public_libs), " public libs")

libs_data = lc.get_libraries(public_libs)

Found:  35  public libs


In [19]:
# Now create emulator with all needed info
em = Emulator(config)

em.set_rand_seed(rand_seed)
em.set_prev_blocks_info(prev_block_data)
em.set_libs(libs_data)

In [20]:
for tx in whitelist_txs:
    # Get current transaction LT & now
    current_tx = tx.transaction

    tx_tlb = Transaction()
    tx_tlb = tx_tlb.cell_unpack(current_tx, True)

    lt = tx_tlb.lt
    now = tx_tlb.now
    
    # Get current trasaction in_msg
    current_tx_cs = current_tx.begin_parse()
    tmp = current_tx_cs.load_ref(as_cs=True)

    if tmp.load_bool():
        in_msg = tmp.load_ref()
    else:
        # TICK TOCK
        in_msg = None

    if in_msg is None:
        success = em.emulate_tick_tock_transaction(
                    account_state,
                    tx_tlb.description.is_tock,
                    now,
                    lt
                )
    else:
        # Emulate
        success = em.emulate_transaction(
            account_state,
            in_msg,
            now,
            lt
        )
        
    assert success
    
    # Emulation transaction equal current transaction
    assert em.transaction.get_hash() == current_tx.get_hash()
    print(f"Emulate {current_tx.get_hash()} success")
    
    # Update account state, go to next transaction
    account_state = em.account.to_cell()

Emulate DCA82F938264969E0A1C6D3DD6BE1E879260D8D17A1831F416DA3424FEC05ABD success
Emulate 5BF39853599F46BDE7E9845C701A74A437F59A0AF7771B030E6E968729FBD77E success
Emulate E7D884136024FC66956E2D55AD321673DCABDBF906B5B3D4F7BB41026284AC44 success
Emulate 895AD56272482D3C832D6C04735442606054E0CA2F407820BFCBC82FC10134F1 success
Emulate 7632D2383D419FD6F025128CA6D3AE61EF340813555CE91BED934086ADB147F4 success
Emulate A0633E1620986FF573EA0AFA167A0AE9CD4059C39C926F3C6FC7AAA42CE7F997 success
Emulate 6E42101DD5FDA37FA82DAB1417AB1DCE1C1B74B6A09AC7280F83E58A1DE95A48 success
Emulate BD23D11FDAE73E64BDD7E160638629F793A8690A06F8487AD024C021843F588B success
Emulate 9F0AB5E4E29A13DB61DEF67F7FA12D09F247D8BC9C895ED31678577B5F4D0C52 success
Emulate F33DD2FCB29C4721A14C5E4782A12B9BDC5A75BF7D3088674BDAB4BC9D34E11F success
Emulate EF4E27E96A447AF2B0EE51677E3B4CBA67BA89C45884155BF4A290E547054F22 success
Emulate E7CBCD016B6A9177F46DFD245F48C568DCA64C20EF1E6AA17B756DC23A803DED success
Emulate D36318CC0A98F0E0683D